In [1]:
from os.path import basename, splitext
import h5py
from PIL import Image
import numpy as np

Image.MAX_IMAGE_PIXELS = None

# Combining Patchs

In [2]:
def combine_h5_files(input_paths, output_path):
    output_file = h5py.File(output_path, 'a')

    for input_path in input_paths:
        i = int(splitext(basename(input_path))[0][-1])
        input_file = h5py.File(input_path, 'r')
        
        for key in input_file.keys():
            input_data = input_file[key][...]
            new_input_data = []
            for x, y in input_data:
                # since we have divided the image in 4 parts
                new_input_data.append(np.array([x + i * 21671, y]))
            new_input_data = np.array(new_input_data)
            data_shape = new_input_data.shape
            
            if key in output_file:
                dset = output_file[key]
                dset.resize(len(dset) + data_shape[0], axis=0)
                dset[-data_shape[0]:] = new_input_data
            else:
                data_type = new_input_data.dtype
                chunk_shape = (1, ) + data_shape[1:]
                maxshape = (None, ) + data_shape[1:]
                dset = output_file.create_dataset(key, shape=data_shape, maxshape=maxshape, chunks=chunk_shape, dtype=data_type)
                dset[:] = new_input_data
                
            for attr_key, attr_val in input_file[key].attrs.items():
                dset.attrs[attr_key] = attr_val
        
        input_file.close()

    output_file.close()
    return output_path

In [ ]:
file_list = [f"/home/mxn2498/projects/search_comparision/sish_new_test/DATA/PATCHES/unknown/unknown/20x/patches/slide2_{i}.h5" for i in range(4)]
output = "/home/mxn2498/projects/search_comparision/sish_new_test/DATA/PATCHES/unknown/unknown/20x/patches/slide2.h5"
combine_h5_files(file_list, output)

In [ ]:
for file in file_list:
    with h5py.File(file, 'r') as f:
        # Copy all groups and datasets from the current file to the output file
        for name in f:
            print(f[name])
            
with h5py.File(output, 'r') as f:
    # Copy all groups and datasets from the current file to the output file
    for name in f:
        print(f[name])

# Combining Masks

In [ ]:
# images = []
# sizes = []
# for i in range(4):
#     path = f"/home/mxn2498/projects/search_comparision/retccl/DATA/PATCHES/unknown/unknown/20x/masks/slide2_{i}.jpg"
#     image = Image.open(path)
#     images.append(image)
#     sizes.append(image.size)

# new_image = Image.new('RGB', (sizes[0][0], sum([sizes[i][1] for i in range(4)])))
# for i in range(4):
#     new_image.paste(images[i], (0, sum([sizes[i][1] for j in range(i)])))

# save_path = f"/home/mxn2498/projects/search_comparision/retccl/DATA/PATCHES/unknown/unknown/20x/masks/slide2.jpg"
# new_image.save(save_path, "JPEG")